# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.768463 ,  2.3276033, -3.7457774, -1.8541754, -5.1695833,
         7.6670885,  2.7118316,  8.495609 ,  1.7038484,  1.1884269],
       [ 4.647688 ,  8.37788  , -9.070581 ,  9.459332 ,  8.450422 ,
        -1.0210547,  3.392087 , -7.8629856, -0.7527662,  0.4838412],
       [-2.9414442,  4.6401706, -4.5027533,  2.2855108,  1.644645 ,
        -2.4937892, -5.2241607, -1.5499196, -8.063638 ,  2.816936 ],
       [-4.271077 ,  5.5611653, -5.6640916, -1.8229511, -9.2925   ,
         0.730283 ,  4.4586773, -2.8876226, -5.1257744,  9.694357 ],
       [ 5.583742 , -4.1515303,  4.369667 , -3.00205  ,  3.6388965,
        -4.341912 , -3.3187115,  6.503671 , -6.865036 , -1.0266498]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:42847 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.809874  ,  8.42267   , -9.23902   ,  9.379147  ,  8.499882  ,
        -1.0592816 ,  3.3437858 , -7.8026114 , -0.5946332 ,  0.26447594,
         5.5073967 , -4.1069794 ,  4.2890773 , -2.8172054 ,  3.6150153 ,
        -4.161299  , -3.6209638 ,  6.2185297 , -6.946047  , -1.0828307 ,
        -5.8267713 ,  2.225877  , -3.8601208 , -1.6974078 , -5.3134174 ,
         7.5795784 ,  2.9187477 ,  8.540422  ,  1.5523204 ,  1.0841805 ],
       [-2.8941855 ,  4.47419   , -4.447567  ,  2.382099  ,  1.7478831 ,
        -2.5046248 , -5.208331  , -1.6937691 , -8.134754  ,  2.64683   ,
        -4.316363  ,  5.5655394 , -5.7322    , -1.7384955 , -9.344655  ,
         0.7084657 ,  4.435839  , -2.9009006 , -4.9486365 ,  9.695302  ,
         8.366521  , -6.247454  , -6.349474  ,  1.9546973 ,  4.1576157 ,
        -9.167903  ,  4.607068  ,  8.788585  ,  6.864424  ,  2.231988  ],
       [-4.665715  , -9.558958  ,  6.657229  ,  4.440129  ,  2.1730294 ,
         2.5904036 ,  0.5800098 ,  6.2550354 , -8